In [1]:
import sys
import os
from pathlib import Path

# importディレクトリの追加
# sys.path.append(os.path.join(os.path.dirname(__file__), '..'))
print(sys.path)

# プロキシの設定
# os.environ['HTTP_PROXY'] = ''
# os.environ['HTTPS_PROXY'] = ''

%matplotlib inline

['/home/y-katayama/notebooks/dl_study/02_pytorch_tutorial', '/usr/lib/python38.zip', '/usr/lib/python3.8', '/usr/lib/python3.8/lib-dynload', '', '/home/y-katayama/venv/pt1.7/lib/python3.8/site-packages']


In [2]:
!nvidia-smi

Wed Dec 14 17:33:33 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.60.13    Driver Version: 525.60.13    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:01:00.0 Off |                  N/A |
| 40%   32C    P8    17W / 184W |   1076MiB /  8192MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  On   | 00000000:03:00.0 Off |                  N/A |
| 40%   

# Autograd(自動微分)

- autogradを使用する場合, **計算グラフでネットワークの順伝播の経路を定義する**
- あとは省略

## Pytorch: 新しいautograd関数の定義

- autograd演算子はTensor上で動作する2つの関数を備える
    - `forward()`: 順伝播処理. 入力Tensorからの出力Tensorを計算
    - `backward()`: 逆伝播処理あるスカラー値の出力Tensorの勾配を受け取り, 同じスカラー値に対応する入力Tensorの勾配を計算する

- `torch.autograd.Function`を継承したサブクラスを定義して, foward, backward()をオーバーライドすることで簡単に独自のautograd演算子を定義することができる

In [8]:
import torch

class MyReLU(torch.autograd.Function):
    """torch.autograd.Functionをサブクラス化して, 
    forward()とbackward()を定義して, 独自のautograd Functionを実装する
    """
    
    @staticmethod
    def forward(ctx, input):
        """順伝播経路では入力Tensorを受け取り, 出力Tensorを返す
        ctxは逆伝播の際に必要な情報を格納するコンテキストオブジェクト
        ctx.save_for_backward()メソッドを使用すると, 
        渡されたデータを保持して逆伝播時に使用できる
        """
        ctx.save_for_backward(input)
        return input.clamp(min=0)
    
    @staticmethod
    def backward(ctx, grad_output):
        """
        逆伝播経路では, 出力に対する損失の勾配を含むTensorを受け取り
        入力に対する損失の勾配を計算する必要がある
        """
        
        # 順伝播時に保持していたTensorを呼び出す
        input, = ctx.saved_tensors
        
        # 下位レイヤから渡された勾配tensorを複製
        grad_input = grad_output.clone()
        
        grad_input[input < 0] = 0
        return grad_input

In [10]:
dtype = torch.float
device = torch.device('cpu')

# N：バッチサイズ         D_in：入力層の次元数
# H：隠れ層の次元数       D_out： 出力層の次元数
N, D_in, H, D_out = 64, 1000, 100, 10

# 乱数により入力データと目標となる出力データを表すTensorを生成
x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)

# 乱数による重みを表すTensorの定義
w1 = torch.randn(D_in, H, device=device, dtype=dtype, requires_grad=True)
w2 = torch.randn(H, D_out, device=device, dtype=dtype, requires_grad=True)

learning_rate = 1e-6
for t in range(500):
    # 関数を適用するには、Function.applyメソッドを用います。
    # reluと命名しておきます。
    relu = MyReLU.apply

    # 順伝播：独自のautograd操作を用いてReLUの出力を算出することで予想結果yを計算します。
    y_pred = relu(x.mm(w1)).mm(w2)

    # 損失の計算と表示
    loss = (y_pred - y).pow(2).sum()
    if t % 100 == 99:
        print(t, loss.item())

    # autogradを利用して逆伝播を実施
    loss.backward()

    # 確率的勾配降下法を用いた重みの更新
    with torch.no_grad():
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad

        # 重みの更新後、手動で勾配を0に初期化
        w1.grad.zero_()
        w2.grad.zero_()

99 388.609375
199 1.6020762920379639
299 0.01111598126590252
399 0.00026563520077615976
499 4.242756403982639e-05


# nnモジュール

- nnパッケージはニューラルネットワークのレイヤーとほぼ同等なモジュールのセットを定義している
- 他にも損失関数等の便利関数群も含まれる
- torch.nnはTensorflowでのKeras, TensorFlow-Slim, TFLearnに相当するパッケージ


In [12]:
import torch

# N：バッチサイズ         D_in：入力層の次元数
# H：隠れ層の次元数       D_out： 出力層の次元数
N, D_in, H, D_out = 64, 1000, 100, 10

x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

# nnパッケージを利用して, レイヤーの連なりとしてモデルを定義
# nn.Sequentialは他のモジュールとして並べて定義することで、各レイヤを順番に実行して出力を得る
# nn.Linearは入力から出力を線形変換する. 重みとバイアスを保持する
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out)
)

# 損失関数はMSEを使用する
loss_fn = torch.nn.MSELoss(reduction='sum')

learning_rate = 1e-4
for t in range(500):
    
    # 順伝播処理
    # Pythonのモジュールオブジェクトを継承しているため__call__が呼びだされる
    
    y_pred = model(x)
    
    # 損失の計算と表示
    loss = loss_fn(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())
        
    # 逆伝播の前に勾配を0フィル
    model.zero_grad()
    
    # 逆伝播
    # 内部ではrequires_grad=TrueとなっているすべてのTensorにモデルのパラメータが保持されている
    # モデルが持つ学習可能なパラメータの勾配を計算できる
    loss.backward()
    
    # SGDを用いた重みの更新
    # 各々のパラメータはTensorなので, これまでと同じ方法で勾配を参照することができる
    
    with torch.no_grad():
        for param in model.parameters():
            param -= learning_rate * param.grad

99 2.690643072128296
199 0.04399355873465538
299 0.0015361031983047724
399 7.08393199602142e-05
499 4.083602561877342e-06


In [13]:
# N：バッチサイズ         D_in：入力層の次元数
# H：隠れ層の次元数       D_out： 出力層の次元数
N, D_in, H, D_out = 64, 1000, 100, 10

# 乱数により入力データと目標となる出力データを表すTensorを生成
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

# nnパッケージを用いてモデルと損失関数を定義
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
)
loss_fn = torch.nn.MSELoss(reduction='sum')

# optimパッケージでモデルの重みを更新するオプティマイザを定義する
# optimパッケージには多くの最適化アルゴリズムが存在するが, ここではAdamを使う
# Adamのコンストラクタの最初の引数により、オプティマイザがどのTensorを更新するか指定できる。
learning_rate = 1e-4
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
for t in range(500):
    # 順伝播:入力xから予測値yをモデルで算出します。。
    y_pred = model(x)

    # 損失の計算と表示
    loss = loss_fn(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())

    # 逆伝播に入る前に、更新されることになる変数（モデルの学習可能な重み）の勾配を
    # optimaizerを使用して0に初期化します。
    # これは、デフォルトで.backward()が呼び出される度に勾配がバッファに蓄積されるため
    # 必要になる操作です（上書きされるわけではない）。
    # 詳しくはtorch.autograd.backwardのドキュメントを参照してください。
    optimizer.zero_grad()

    # 逆伝播：モデルのパラメータに対応する損失の勾配を計算
    loss.backward()

    # オプティマイザのstep関数を呼び出すことでパラメータを更新
    optimizer.step()

99 41.6004638671875
199 0.4085378646850586
299 0.005604689475148916
399 9.623980440665036e-05
499 8.11555935342767e-07


# nn Modulesの改変

- 既存のモジュールにある一連の演算処理を改変したい場合, `nn.Module`をサブクラス化し, 
  他のモジュールやTensorの他のautograd操作を利用して, 入力Tensorや出力Tensorを生成するforward()メソッドを定義することで, ユーザ独自のモジュールを作ることができる
    

In [16]:
import torch


class TwoLayerNet(torch.nn.Module):
    def __init__(self, D_in, H, D_out):
        """
        コンストラクタにより2つのnn.Linearモジュールをインスタンスとし定義し
        それらをメンバ変数に割り当てます。
        """
        super(TwoLayerNet, self).__init__()
        self.linear1 = torch.nn.Linear(D_in, H)
        self.linear2 = torch.nn.Linear(H, D_out)

    def forward(self, x):
        """
        forward関数は入力データのTensorを受け入れ、出力データのTensorを返します。
        Tensorの任意の演算子と同様に、コンストラクタで定義されたモジュールを使用できます。
        """
        h_relu = self.linear1(x).clamp(min=0)
        y_pred = self.linear2(h_relu)
        return y_pred


# N：バッチサイズ         D_in：入力層の次元数
# H：隠れ層の次元数       D_out： 出力層の次元数
N, D_in, H, D_out = 64, 1000, 100, 10

# 乱数により入力データと目標となる出力データを表すTensorを生成
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

# 上で定義したクラスをインスタンス化してモデルを構築
model = TwoLayerNet(D_in, H, D_out)

# 損失関数とオプティマイザを定義します。
# model.parameters()を呼び出すことで、モデルのメンバ変数である2つのnnn.Linearモジュールの
# 学習可能なパラメータをSGDのコンストラクタの引数として渡すことができます。
criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4)
for t in range(500):
    # 順伝播:入力xから予測値yをモデルで算出します。
    y_pred = model(x)

    # 損失の計算と表示
    loss = criterion(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())

    # 勾配を0に初期化し、逆伝播を実行することで重みを更新
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


99 2.353790521621704
199 0.04485118389129639
299 0.0015129691455513239
399 6.243730604182929e-05
499 2.8527128961286508e-06


In [17]:
# -*- coding: utf-8 -*-
import random
import torch


class DynamicNet(torch.nn.Module):
    def __init__(self, D_in, H, D_out):
        """
        コンストラクタにより、順伝播で使用する3層のnn.Linearのインスタンスを定義します。
        """
        super(DynamicNet, self).__init__()
        self.input_linear = torch.nn.Linear(D_in, H)
        self.middle_linear = torch.nn.Linear(H, H)
        self.output_linear = torch.nn.Linear(H, D_out)

    def forward(self, x):
        """
        順伝播の経路の実装では、ランダムに0から3までの値を選択し、
        その数だけ中間層のモジュールを再利用することで、隠れ層の出力を計算します。

        それぞれの順伝播の経路を表す計算グラフは動的に変化するので、
        繰り返しや、条件分岐といったPythonの標準的なフロー制御を利用して
        順伝播の経路を定義することができます。

        この結果から確認できるように、計算グラフを定義する際に、問題なく何度も同じ
        モジュールを使いまわすことができるのは、一度しかモジュールを利用することが
        できなったLua Torchから大きく改善したところと言えます。
        """
        h_relu = self.input_linear(x).clamp(min=0)
        for _ in range(random.randint(0, 4)):
            h_relu = self.middle_linear(h_relu).clamp(min=0)
        y_pred = self.output_linear(h_relu)
        return y_pred


# N：バッチサイズ         D_in：入力層の次元数
# H：隠れ層の次元数       D_out： 出力層の次元数
N, D_in, H, D_out = 64, 1000, 100, 10

# 乱数により入力データと目標となる出力データを表すTensorを生成
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

# 上で定義したクラスをインスタンス化してモデルを構築します。
model = DynamicNet(D_in, H, D_out)

# 損失関数とオプティマイザを定義します。
# この奇妙なモデルを通常の確率勾配降下法で訓練するのは難しいので、モーメンタムを使用します。
criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4, momentum=0.9)
for t in range(500):
    # 順伝播:入力xから予測値yをモデルで算出します。
    y_pred = model(x)

    # 損失の計算と表示
    loss = criterion(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())

    # 勾配を0に初期化し、逆伝播を実行することで重みを更新します
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

99 83.9114761352539
199 12.065067291259766
299 10.240217208862305
399 0.3152943253517151
499 0.5636540055274963
